In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mongoCRUD import AnimalShelter

#########################
# CSS Style for Dashboard
#########################

LOGO = {'diplay': 'block', 'width': '10%', 'margin': 'auto'}

###########################
# Filters for dropdown menu
###########################

FILTER_ALL = {}

FILTER_WATER = {'$or':
                [{"breed":"Labrador Retriever Mix"},
                 {"breed":"Chesapeake Bay Retriever"},
                 {"breed":"Newfoundland"}
                ],
                "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                "sex_upon_outcome":"Intact Female"}

FILTER_MOUNTAIN = {'$or':                 
                   [{"breed":"German Shepherd"},
                    {"breed":"Alaskan Malamute"},
                    {"breed":"Old English Sheepdog"},
                    {"breed":"Siberian Husky"},
                    {"breed":"Rottweiler"}
                   ],
                   "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                   "sex_upon_outcome":"Intact Male"}

FILTER_DISASTER = {'$or':
                   [{"breed":"Doberman Pinscher"},
                    {"breed":"German Shepherd"},
                    {"breed":"Golden Retriever"},
                    {"breed":"Bloodhound"},
                    {"breed":"Rottweiler"}
                   ],
                   "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                   "sex_upon_outcome":"Intact Male"}

FILTER_CAT = {"animal_type": "Cat"}

###########################
# Data Manipulation / Model
###########################

# Username and Password to access Database
username = "aacuser"
password = "newuserPassword"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Populate data frame
cursor = db.read(FILTER_ALL)
df = pd.DataFrame.from_records(list(cursor))
#df[' index'] = range(1, len(df) + 1)
#df = pd.DataFrame.from_records(db.read({}))
#df.drop(columns=['_id'],inplace=False)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
        
# Header
    html.Div([
        html.Center(html.B(html.Img(src='data:imag/png;base64,{}'.format(encoded_image.decode()), style = LOGO))),
        html.Center(html.B(html.H3('Bret Thex - CS-340 Dashboard')))
    ]),
    html.Hr(),
    html.Div(
        
# Dropdown Menu
    dcc.Dropdown(
        id = 'filter-dropdown',
       options=[
           {'label': 'All (Reset)', 'value': 'All'},
           {'label': 'Water Rescue', 'value': 'Water'},
           {'label': 'Mountian Rescue', 'value': 'Mountain'},
           {'label': 'Disaster Rescue', 'value': 'Disaster'},
   ],
   value='All',
)
    ),
    html.Hr(),
    
# Data Table
    html.Div(id = 'table-output', children = [
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            )]),
    #html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        # Graph
        html.Center(html.B(html.Div(
            dcc.Graph(id='pie-chart'),
            className='col s12 m6',

            ))),
        html.Center(html.B(html.Div(
            style={'max-height': '500px'},
            id='map-id',
            className='col s6 m6',
            )))
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


###################
# Dropdown Callback
###################
@app.callback(
    Output('table-output', 'children'),
    Input('filter-dropdown', 'value')
    )
def update_output(value):
    
    # Generate data frame based on dropdown selection
    if value == 'All':        
        df = pd.DataFrame.from_records(db.read(FILTER_ALL))   
    elif value == 'Water':        
        df = pd.DataFrame.from_records(db.read(FILTER_WATER))      
    elif value == 'Mountain':      
        df = pd.DataFrame.from_records(db.read(FILTER_MOUNTAIN))      
    elif value == 'Disaster':
        df = pd.DataFrame.from_records(db.read(FILTER_DISASTER))
    
    return [
        dash_table.DataTable(
            id = 'datatable-id',
            columns = [{"name": i, "id": i} for i in df.columns],
            data = df.to_dict('records'),
            row_selectable="single",
            row_deletable=False,
            selected_rows=[0],
            page_action = 'native',
            page_current = 0,
            page_size = 6) 
    ]
    
################
# Graph Callback
################

@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(dff,names='outcome_type')
    return fig

########################
# Row Selection Callback
########################
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

##############
# Map Callback
##############

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
               center=[dff.iloc[row,13],dff.iloc[row,14]],
               zoom=10, 
               children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



#app.run_server(debug=False)
app.run_server()


Initialization successful
Dash app running on http://127.0.0.1:10826/
